<center>
<img src="https://tensorflowkorea.files.wordpress.com/2020/12/4.-e18492e185a9e186abe1848ce185a1-e18480e185a9e186bce18487e185aee18492e185a1e18482e185b3e186ab-e18486e185a5e18489e185b5e186abe18485e185a5e18482e185b5e186bce18483e185b5e186b8e18485e185a5e.png?w=972" width="200" height="200"><br>
</center>

# Chapter.9 텍스트를 위한 인공 신경망

## 09-1 순차 데이터와 순차 신경망

### - 순차 데이터

순차 데이터는 텍스트나 시계열 데이터와 같이 순서에 의미가 있는 데이터를 말한다. 예를 들면 일별 온도를 기록한 데이터에서 날짜 순서를 뒤죽박죽 섞는다면 내일의 온도를 쉽게 예상하기 어렵다. 지금까지 우리가 보았던 데이터는 순서와는 상관이 없었다. 예를 들어 패션 MNIST 데이터를 생각해보자. 이 데이터를 신경망 모델에 전달할 때 샘플을 랜덤하게 섞은 후 훈련 세트와 검증 세트로 나누었다. 즉 샘플의 순서는 상관이 없었다. 심지어 골고루 섞는 편이 결과가 더 좋았다.

이 장에서 사용하려는 댓글, 즉 텍스트 데이터는 단어의 순서가 중요한 순차 데이터이다. 이런 데이터는 순서를 유지하며 신경망에 주입해야 한다.따라서 순차 데이터를 다룰 때는 이전에 입력한 데이터를 기억하는 기능이 필요하다. 예를 들어 '별로지만 추천해요' 에서 '추천해요'가 입력될 때 '별로지만'을 기억하고 있어야 이 댓글을 무조건 긍정적이라고 판단하지 않을 것이다.

완전 연결 신경망이나 합성곱 신경망은 이런 기억 장치가 없다. 하나의 샘플을 사용하여 정방향 계산을 수행하고 나면 그 샘플은 버려지고 다음 샘플을 처리할 때 재사용하지 않는다. 이렇게 입력 데이터의 흐름이 앞으로만 전달되는 신경망을 피드포워드 신경망이라고 한다. 이전에 배웠던 완전 연결 신경망과 합성곱 신경망이 모두 피드포워드 신경망이다.

신경망이 이전에 처리했던 샘플을 다음 샘플을 처리하는데 재사용하기 위해서는 이렇게 데이터 흐름이 앞으로만 전달 되어서는 안된다. 다음 샘플을 위해 이전 데이터가 신경망 층에 순환될 필요가 있다.

### - 순환 신경망

순환 신경망은 일반적인 완전 연결 신경망과 거의 비슷하다. 완전 연결 신경망에 이전 데이터의 처리 흐름을 순환하는 고리 하나만 추가하면 된다. 순환 신경망에서는 '이전 샘플에 대한 기억을 가지고 있다'고 종종 말한다. 이렇게 샘플을 처리하는 한 단계를 타임스텝이라고 한다. 순환 신경망은 이전 타임스텝의 샘플을 기억하지만 타임스텝이 오래될수록 순환되는 정보는 희미해진다.

순환 신경망에서는 특별히 층을 셀이라고 부른다. 한 셀에는 여러 개의 뉴런이 있지만 완전 연결 신경망과 달리 뉴런을 모두 표시하지 않고 하나의 셀로 층을 표현한다. 또 셀의 출력을 은닉 상태라고 부른다. 일반적으로 은닉층의 활성화 함수로는 하이퍼볼릭 탄젠트 함수가 많이 사용된다. tanh 함수도 S자 모양을 띠기 때문에 종종 시그모이드 함수라고 부르기도 한다. tanh 함수는 시그모이드 함수와 달리 -1 ~ 1 사이의 범위를 가진다. 

다른 신경망과 마찬가지로 순환 신경망에도 활성화 함수가 반드시 필요하다. 순환 신경망의 뉴런은 가중치가 하나 더 있다. 바로 이전 타임스텝의 은닉 상태에 곱해지는 가중치이다. 셀은 입력과 이전 타임스텝의 은닉 상태를 사용하여 현재 타임스텝의 은닉 상태를 만든다.

### - 셀의 가중치와 입출력

순환 신경망의 셀에서 필요한 가중치 크기를 계산해 보자. 복잡한 모델을 배울수록 가중치 개수를 계산해 보면 잘 이해하고 있는지 알 수 있다. 만약 순환층에 입력되는 특성의 개수가 4개이고 순환층의 뉴런이 3개라고 가정해 보자.

wx의 크기는 입력층과 순환층의 뉴런이 모두 연결되기 때문에 가중치 wx의 크기는 4 x 3 = 12가 된다. 그럼 순환층에서 다음 타임스텝에 재사용되는 은닉 상태를 위한 가중치 wh의 크기는 어떻게 될까? 순환층에 있는 첫 번째 뉴런의 은닉 상태가 다음 타임스텝에 재사용될 때 첫 번째 뉴런과 두 번쨰 뉴런, 세 번째 뉴런에 모두 전달된다. 따라서 이 순환층에서 은닉상태를 위한 가중치 wh는 3 x 3 = 9개 이다.

이번에는 파라미터의 개수를 계산해보자. 가중치에 절편을 더하면 된다. 여기엔 각 뉴런마다 하나의 절편이 있다. 따라서 이 순환층은 모두 12 + 9 + 3 = 24개의 모델 파라미터를 가지고 있다.

순환층은 일반적으로 샘플마다 2개의 차원을 가진다. 보통 하나의 샘플을 하나의 시퀀스라고 말한다. 시퀀스 안에는 여러 개의 아이템이 들어있다. 여기에서 시퀀스의 길이가 바로 타임스텝의 길이가 된다. 하나의 샘플은 시퀀스 길이와 단어 표현의 2차원 배열이다. 순환층을 통과하면 1차원 배열로 바뀐다. 이 1차원 배열의 크기는 순환층의 뉴런 개수에 의해 결정된다. 사실 순환층은 기본적으로 마지막 타임스텝의 은닉 상태만 출력으로 내보낸다. 이는 마치 입력된 시퀀스 길이를 모두 읽어서 정보를 마지막 은닉 상태에 압축하여 전달하는 것처럼 볼 수 있다. 이제 순환 신경망이 정보를 기억하는 메모리를 가진다고 표현하는지 이해할 수 있다. 또 순환 신경망이 순차 데이터에 잘 맞는 이유를 파악할 수 있다.

순환 신경망도 완전 연결망이나 합성곱 신경망처럼 여러 개의 층을 쌓을 수 있다. 순환층을 여러 개 쌓았을 때는 셀의 출력은 어떻게 달라질까? 셀의 입력은 샘플마다 타임스텝과 단어 표현으로 이루어진 2차원 배열이어야 한다.

출력층의 구성에 대해 알아보자. 합성곱 신경망과 마찬가지로 순환 신경망도 마지막에는 밀집층을 두어 클래스를 분류한다. 다중 분류일 경우에는 출력층에 클래스 개수만큼 뉴런을 두고 소프트맥스 활성화 함수를 사용한다. 이진 분류일 경우에는 하나의 뉴런을 두고 시그모이드 활성화 함수를 사용한다. 합성곱 신경망과 다른 점은 마지막 셀의 출력이 1차원이기 때문에 Flatten 클래스로 펼칠 필요가 없다. 셀의 출력을 그대로 밀집층에 사용할 수 있다.

## 09-2 순환 신경망으로 IMDB 리뷰 분류하기

## - IMDB 리뷰 데이터셋


IMDB 리뷰 데이터셋은 유명한 인터넷 영화 데이터베이스인 imdb.com에서 수집한 리뷰를 감상평에 따라 긍정과 부정으로 분류해 놓은 데이터셋이다. 총 50,000개의 샘플로 이루어져 있고 훈련 데이터와 테스트 데이터에 각각 25,000개씩 나누어져 있다. 사실 텍스트 자체를 신경망에 전달하지는 않는다. 컴퓨터에서 처리하는 모든 것은 어떤 숫자 데이터이다. 앞서 합성곱 신경망에서 이미지를 다룰 때는 특